# Applied Data Science Capstone Week 3

## Question 1
**(10 Marks)**

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, [https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M), in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

![Unable to Load Image](img1.png)

3. To create the above dataframe:

    - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
    - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


4. Submit a link to your Notebook on your Github repository. **(10 marks)**

In [3]:
# Settle the Imports 
import requests
import pandas as pd
import numpy as np

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Map Plotting
#!conda install -c conda-forge folium=0.5.0 --yes
import folium 

#!conda install -c anaconda lxml --yes
import lxml

print("Hello Capstone Project Course!")

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Alkaff\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be UPDATED:

  ca-certificates      anaconda::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.2-hecda079_0
  certifi               anaconda::certifi-2020.4.5.1-py37_0 --> conda-forge::certifi-2020.4.5.2-py37hc8dfbb8_0
  conda                        anaconda::conda-4.8.3-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1
  openssl               anaconda::openssl-1.1.1c-he774522_1 --> conda-forge::openssl-1.1.1f-hfa6e2cd_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Alkaff\Anaconda3

  added / updated specs:
    - lxml


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates

### Explanation before code

First, we download the data and directly store into a variable <code>raw_tables</code>. The <code>pd.read_html()</code> will return a list of tables. We set <code>header=0</code> because we are going to define our own headers. 

The variable <code>raw_tables</code> contains a list of dataframes where each dataframe is actually a pandas dataframe. We run a for loop through the list of dataframes until the number of columns match the number of headings we generated. 

In [11]:
# Download Data and Store into dataframe

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

raw_tables = pd.read_html(url, header=0, keep_default_na=False)
headings = ['Postal Code', 'Borough', 'Neighborhood']

for tbl in raw_tables:
    print(type(tbl))
    current_headings = tbl.columns.values[:3]
    if len(current_headings) != len(headings):
        continue
    if all(current_headings == headings):
        break

tbl.head()

<class 'pandas.core.frame.DataFrame'>


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Explanation before code

We define the <code>postal_df</code> to be the dataframe that contain the same data as the dataframe <code>tbl</code> but without the postal codes that are "Not Assigned". Remember, we have not filtered out the postal codes that are "Not Assigned". We are going to do that with a simple line of code as below. 

In [13]:
# Remove the "Not Assigned" codes

postal_df = tbl[tbl.Borough != 'Not assigned']
postal_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
# Checking the dimensions of table
postal_df.shape

(103, 3)

## Question 2 
**(2 Marks)**

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

![Unable to Load Image](img2.png)

**Important Note:** There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. **(2 marks)**

### Explanation before code

We download the data directly from the url (the CSV file). We use <code>pd.read_csv()</code> since its a CSV file and store the dataframe into <code>coord_df</code>.

In [15]:
# Load the Dataframe from the given url
url="http://cocl.us/Geospatial_data"
coord_df = pd.read_csv(url)
coord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Explanation before code

We are going to do something similar to SQL Inner Join for merging the 2 dataframes. This is done using <code>pd.merge()</code> where:

1. <code>left</code> is the left side of the table (our case will be <code>postal_df</code>) 
2. <code>right</code> will be the right side of the table (our case will be <code>coord_df</code>) 
3. <code>left_on</code> and <code>right_on</code> will be the index column on the respective tables to match (in our case, both table's index column for matching would be <code>Postal Code</code>)

In [16]:
# Merge both dataframes 
merged_df = pd.merge(left=postal_df, right=coord_df, left_on='Postal Code', right_on='Postal Code')
merged_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Question 3
#### (3 Marks)

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. **(3 marks)**

### Explanation before code

Since we only have 100+ records, we can apply the <code>unique()</code> function to get the list of places. From there, we select the required places and store it into a new list (<code>selected_list</code>)

**Note:**
We are doing this shortcut because the dataset is small. For larger datasets, we will iterate through the list of unique places and use the <code>find()</code> function to look for the keywords. Then we will filter and store those keywords into <code>selected_list</code>. 

In [21]:
places_list = list(merged_df["Borough"].unique())
print(places_list)

['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto', 'Central Toronto', 'Mississauga']


### Explanation before code

We have the list of places to filter. Now we will filter from <code>merged_df</code> and store into a new variable <code>toronto_df</code>

In [23]:
selected_list = ['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

toronto_df = merged_df.loc[merged_df["Borough"].isin(selected_list)]
print(toronto_df.shape)
toronto_df.head()



(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Explanation before code

We will initialise the map and the colour list (<code>color_list</code>). Each Borough will be of a specific colour. Also, the coordinates for Toronto would be: 

*43.6532° N, 79.3832° W*

In [47]:
color_list = {'Downtown Toronto':'#ffb805', 
              'East Toronto':'#2fff05', 
              'West Toronto':'#05ffea', 
              'Central Toronto':'#ff05d5'}

map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

### Explanation before code

The map of Toronto is successfully generated. Now we will add all the markers for visualizing. We will use the for loop to iterate through all the points and add them into the map directly. 

In [48]:
for lat, lon, bor, nh in zip(toronto_df["Latitude"],toronto_df["Longitude"],toronto_df["Borough"],toronto_df["Neighborhood"]):
    lbl = folium.Popup(nh + ' (' + bor + ')', parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=lbl, color=color_list[bor], 
                        fill=True, fill_color=color_list[bor], fill_opacity=0.7).add_to(map_toronto)

map_toronto

# The End